#### Running GEMMA from GN3

This document outlines how to use gemma from Genenetwork3.

If you are running gn3 through a development environment, ensure that it is up and running by running the command:

```sh
env FLASK_APP="main.py" flask run --port 8080
```

To ensure that things are working, run:

In [2]:
import requests
r = requests.get("http://127.0.0.1:8080/gemma/version")
print(r.json())

{'code': 0, 'output': 'gemma-wrapper 0.98.1 (Ruby 2.6.5) by Pjotr Prins 2017,2018\n'}
